In [114]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MovieLens_Recommendations").getOrCreate()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44413)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44413)

In [76]:
ratings = (
    spark.read.csv(
        "ratings.csv",
        sep=",",
        header=True,
        quote='"',
        schema="userId INT, movieId INT, rating DOUBLE, timestamp INT",
    )
    .drop(col("timestamp"))
    .withColumnRenamed("userId", "user")
    .withColumnRenamed("movieId", "item")
)

In [77]:
ratings.show()
ratings.count()

+----+----+------+
|user|item|rating|
+----+----+------+
|   1|   1|   4.0|
|   1|   3|   4.0|
|   1|   6|   4.0|
|   1|  47|   5.0|
|   1|  50|   5.0|
|   1|  70|   3.0|
|   1| 101|   5.0|
|   1| 110|   4.0|
|   1| 151|   5.0|
|   1| 157|   5.0|
|   1| 163|   5.0|
|   1| 216|   5.0|
|   1| 223|   3.0|
|   1| 231|   5.0|
|   1| 235|   4.0|
|   1| 260|   5.0|
|   1| 296|   3.0|
|   1| 316|   3.0|
|   1| 333|   5.0|
|   1| 349|   4.0|
+----+----+------+
only showing top 20 rows



100836

In [28]:
from pyspark.ml.recommendation import ALS

In [ ]:
rank = 7
max_iter = 25
seed = 10

als = ALS(rank=rank, maxIter=max_iter, seed=seed)
model = als.fit(ratings)

In [111]:
item_factors = model.itemFactors.orderBy("id")
user_factors = model.userFactors.orderBy("id")

print("First 30 itemFactors")
item_factors.show(30, False)

print("First 30 userFactors")
user_factors.show(30, False)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:44413)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:44413)

In [83]:
simple_test_data = [
    (1, 2), 
    (10, 3), 
    (10, 8), 
    (20, 4), 
    (9, 7), 
    (550, 235)
]

test = spark.createDataFrame(simple_test_data, "user INT, item INT")

In [84]:
test_prediction = model.transform(test)

In [86]:
test_prediction.show()

+----+----+----------+
|user|item|prediction|
+----+----+----------+
|  10|   3|  1.957693|
| 550| 235| 3.4253366|
|  20|   4| 2.3973246|
|  10|   8| 1.5874846|
|   9|   7| 2.4595492|
|   1|   2| 3.9841352|
+----+----+----------+



In [70]:
predictions_in_pandas = test_prediction.toPandas()

In [87]:
print(predictions_in_pandas)

   user  item  prediction
0    10     3    2.246040
1   550   235    3.208731
2    20     4    2.438197
3    10     8    1.380121
4     9     7    2.920148
5     1     2    4.126770


In [88]:
model.__dir__()

['uid',
 '_paramMap',
 '_defaultParamMap',
 '_params',
 '_java_obj',
 'coldStartStrategy',
 'itemCol',
 'predictionCol',
 'userCol',
 '__module__',
 '__doc__',
 'rank',
 'userFactors',
 'itemFactors',
 'recommendForAllUsers',
 'recommendForAllItems',
 'recommendForUserSubset',
 'recommendForItemSubset',
 '__metaclass__',
 '__init__',
 '_transform',
 '_make_java_param_pair',
 '_transfer_params_to_java',
 '_transfer_param_map_to_java',
 '_create_params_from_java',
 '_transfer_params_from_java',
 '_transfer_param_map_from_java',
 '_empty_java_param_map',
 '_to_java',
 '_from_java',
 'copy',
 '__del__',
 '_create_from_java_class',
 '_call_java',
 '_new_java_obj',
 '_new_java_array',
 '__dict__',
 '__weakref__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir_

In [92]:
model.itemCol

Param(parent='ALS_ed441634ca81', name='itemCol', doc='column name for item ids. Ids must be within the integer value range.')

In [90]:
model.predictionCol

Param(parent='ALS_ed441634ca81', name='predictionCol', doc='prediction column name')

In [91]:
model.userCol

Param(parent='ALS_ed441634ca81', name='userCol', doc='column name for user ids. Ids must be within the integer value range.')

In [94]:
model.itemFactors.show()

+---+--------------------+
| id|            features|
+---+--------------------+
| 10|[-0.6444223, -0.7...|
| 20|[-0.24773549, -0....|
| 30|[-0.40664145, -0....|
| 40|[-0.8642203, -0.5...|
| 50|[-0.5863519, -1.0...|
| 60|[0.15890557, -0.3...|
| 70|[-0.46935442, -0....|
| 80|[-0.61502326, -0....|
|100|[0.024715971, -0....|
|110|[-0.65299803, -0....|
|140|[-0.5847818, -0.7...|
|150|[-0.55277646, -1....|
|160|[-0.027915573, -0...|
|170|[-0.5005498, -1.0...|
|180|[-0.5916557, -0.8...|
|190|[-0.7189717, -0.4...|
|210|[-0.101121746, -0...|
|220|[-0.025304731, -0...|
|230|[-0.17728098, -1....|
|240|[0.004668632, -0....|
+---+--------------------+
only showing top 20 rows



In [95]:
model.userFactors.show()

+---+--------------------+
| id|            features|
+---+--------------------+
| 10|[-0.52024275, -1....|
| 20|[-0.10867017, -1....|
| 30|[-0.49758947, -1....|
| 40|[-0.39843017, -1....|
| 50|[-0.31211424, -0....|
| 60|[-0.6662283, -0.8...|
| 70|[-0.7381246, -0.9...|
| 80|[-0.75811034, -0....|
| 90|[0.025020013, -1....|
|100|[-0.40679854, -0....|
|110|[-0.3105018, -0.9...|
|120|[-0.34617805, -0....|
|130|[-0.12310147, -0....|
|140|[-0.6032331, -0.8...|
|150|[-0.7468541, -0.7...|
|160|[0.208893, -0.679...|
|170|[-0.26220936, -0....|
|180|[-0.3681365, -1.2...|
|190|[-0.5861341, -1.0...|
|200|[-0.27623278, -0....|
+---+--------------------+
only showing top 20 rows

